In [ ]:

!pip -q install ultralytics kagglehub scikit-learn pandas matplotlib pyyaml
from ultralytics import YOLO
import os, sys, glob, shutil, json, random, zipfile, yaml, pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support

print("Environment ready")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Environment ready


In [ ]:

# Detect dataset location or download via KaggleHub
base_candidates = [
    "/kaggle/input/license-plate-dataset",                          # Kaggle environment
    os.path.expanduser("~/.cache/kagglehub/datasets/ronakgohil/license-plate-dataset/versions/latest")
]
dataset_root = None

for cand in base_candidates:
    if os.path.exists(cand):
        dataset_root = cand
        break

if dataset_root is None:
    print("Trying KaggleHub download...")
    import kagglehub
    try:
        # Ensure user is logged in (run kagglehub.login() once manually if needed)
        path = kagglehub.dataset_download("ronakgohil/license-plate-dataset")
        dataset_root = path
        print("Downloaded via KaggleHub:", dataset_root)
    except Exception as e:
        print("Could not download via KaggleHub. Error:", e)
        raise SystemExit("Please run kagglehub.login() and ensure your Kaggle API key is configured.")

print("Dataset root:", dataset_root)

# Some Kaggle datasets come as a subfolder 'archive' with ZIP(s) inside.
archive_dir = os.path.join(dataset_root, "archive")
if os.path.isdir(archive_dir):
    dataset_root = archive_dir  # we'll search zips inside
    print("Detected 'archive' folder:", dataset_root)

# Create a working extraction directory
work_dir = "/kaggle/working/license_plate_data" if os.path.exists("/kaggle/working") else "./license_plate_data"
os.makedirs(work_dir, exist_ok=True)

# Unzip any zip files found under dataset_root (non-destructive re-run safe)
zips = [os.path.join(dataset_root, f) for f in os.listdir(dataset_root) if f.lower().endswith(".zip")]
if len(zips) == 0:
    print("ℹ️ No ZIP files found — data may already be extracted or provided as folders.")
else:
    for z in zips:
        print("Extracting:", z)
        with zipfile.ZipFile(z, 'r') as zip_ref:
            zip_ref.extractall(work_dir)

print("Data prepared in:", work_dir)
print("Dir listing:", os.listdir(work_dir))


Trying KaggleHub download...
Using Colab cache for faster access to the 'license-plate-dataset' dataset.
Downloaded via KaggleHub: /kaggle/input/license-plate-dataset
Dataset root: /kaggle/input/license-plate-dataset
Detected 'archive' folder: /kaggle/input/license-plate-dataset/archive
ℹ️ No ZIP files found — data may already be extracted or provided as folders.
Data prepared in: ./license_plate_data
Dir listing: []


In [ ]:
# ===== Robust dataset resolver for Colab/Kaggle/local =====
import os, sys, shutil, zipfile, glob

def resolve_dataset_root():
    # 1) Типовые кандидаты (вдруг ты реально на Kaggle)
    candidates = [
        "/kaggle/input/license-plate-dataset",
        "/content/kaggle/input/license-plate-dataset",
        os.path.expanduser("~/.cache/kagglehub/datasets/ronakgohil/license-plate-dataset/versions/latest"),
    ]

    for c in candidates:
        if os.path.exists(c):
            print("Found dataset at:", c)
            return c

    # 2) Если ни один путь не найден — качаем KaggleHub
    print("None of the default paths exist. Trying KaggleHub download...")
    try:
        import kagglehub
    except ImportError:
        !pip -q install kagglehub
        import kagglehub

    try:
        # Если не логинился раньше — нужно один раз выполнить:
        # kagglehub.login()
        path = kagglehub.dataset_download("ronakgohil/license-plate-dataset")
        print("Downloaded via KaggleHub:", path)
        return path
    except Exception as e:
        print("KaggleHub download failed:", e)
        raise SystemExit("Configure Kaggle API: run kagglehub.login() once, then rerun this cell.")

def recursive_unzip_or_copy(dataset_root, work_dir):
    os.makedirs(work_dir, exist_ok=True)
    # если есть подкаталог archive — углубимся
    archive_dir = os.path.join(dataset_root, "archive")
    if os.path.isdir(archive_dir):
        dataset_root = archive_dir

    # соберём все ZIP рекурсивно
    zip_paths = []
    for r, d, f in os.walk(dataset_root):
        for name in f:
            if name.lower().endswith(".zip"):
                zip_paths.append(os.path.join(r, name))

    if zip_paths:
        print(f"Found {len(zip_paths)} zip(s). Extracting into:", work_dir)
        for z in zip_paths:
            print("  ->", z)
            with zipfile.ZipFile(z, 'r') as zf:
                zf.extractall(work_dir)
    else:
        print("ℹNo zips found. Copying dataset_root -> work_dir")
        # копируем всё содержимое (read-only -> writable)
        for item in os.listdir(dataset_root):
            src = os.path.join(dataset_root, item)
            dst = os.path.join(work_dir, item)
            if os.path.isdir(src):
                shutil.copytree(src, dst, dirs_exist_ok=True)
            else:
                shutil.copy2(src, dst)

    # Диагностика
    print("work_dir content (top 2 levels):")
    depth0 = work_dir.count(os.sep)
    for r, d, f in os.walk(work_dir):
        depth = r.count(os.sep) - depth0
        print(r, "->", len(f), "files")
        if depth >= 1:
            continue

def assert_images_exist(work_dir):
    img_exts = (".jpg",".jpeg",".png",".bmp",".tif",".tiff")
    img_paths = []
    for r, d, f in os.walk(work_dir):
        for name in f:
            if name.lower().endswith(img_exts):
                img_paths.append(os.path.join(r, name))
    print("Found images:", len(img_paths))
    if len(img_paths) == 0:
        # полезная диагностика
        print("Listing top-level dirs/files under work_dir for inspection:")
        for item in os.listdir(work_dir):
            print(" -", item)
        raise RuntimeError("No images found after extraction. The dataset may have a different nested structure. Inspect the printed tree.")

# ---------- Run ----------
dataset_root = resolve_dataset_root()

work_dir = "/kaggle/working/license_plate_data" if os.path.exists("/kaggle/working") else "./license_plate_data"
recursive_unzip_or_copy(dataset_root, work_dir)
assert_images_exist(work_dir)

print("Dataset ready at:", work_dir)


Found dataset at: /kaggle/input/license-plate-dataset
ℹNo zips found. Copying dataset_root -> work_dir
work_dir content (top 2 levels):
./license_plate_data -> 2 files
./license_plate_data/images -> 0 files
./license_plate_data/images/train -> 1526 files
./license_plate_data/images/val -> 169 files
./license_plate_data/labels -> 0 files
./license_plate_data/labels/train -> 1526 files
./license_plate_data/labels/val -> 169 files
Found images: 1695
Dataset ready at: ./license_plate_data


In [ ]:

def find_yolo_dirs(root):
    # Check for standard YOLO structure
    images_dir = None
    labels_dir = None
    for candidate in ["images", "Images"]:
        p = os.path.join(root, candidate)
        if os.path.isdir(p):
            images_dir = p
            break
    for candidate in ["labels", "Labels"]:
        p = os.path.join(root, candidate)
        if os.path.isdir(p):
            labels_dir = p
            break
    return images_dir, labels_dir

def has_split(images_dir):
    if images_dir is None:
        return False
    sub = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir,d))]
    return set(sub) >= set(["train","val"]) or set(sub) >= set(["train","valid"])

def ensure_split(root):
    # Try to find an existing YOLO structure
    images_dir, labels_dir = find_yolo_dirs(root)
    if images_dir and has_split(images_dir):
        # Normalize "valid" -> "val"
        for old in ["valid", "validation"]:
            oldp = os.path.join(images_dir, old)
            if os.path.isdir(oldp):
                os.rename(oldp, os.path.join(images_dir, "val"))
            oldp = os.path.join(os.path.dirname(images_dir), "labels", old)
            if os.path.isdir(oldp):
                os.rename(oldp, os.path.join(os.path.dirname(images_dir), "labels", "val"))
        # If test not present, make small test from val
        test_dir = os.path.join(images_dir, "test")
        if not os.path.isdir(test_dir):
            os.makedirs(test_dir, exist_ok=True)
            labels_test = os.path.join(os.path.dirname(images_dir), "labels", "test")
            os.makedirs(labels_test, exist_ok=True)
            val_images = glob.glob(os.path.join(images_dir, "val", "*.*"))
            move_count = max(1, len(val_images)//10)
            for p in random.sample(val_images, min(move_count, len(val_images))):
                base = os.path.basename(p)
                # Move image
                shutil.move(p, os.path.join(test_dir, base))
                # Move label if exists
                label_src = os.path.join(os.path.dirname(images_dir), "labels", "val", os.path.splitext(base)[0] + ".txt")
                label_dst = os.path.join(os.path.dirname(images_dir), "labels", "test", os.path.splitext(base)[0] + ".txt")
                if os.path.exists(label_src):
                    shutil.move(label_src, label_dst)
        return images_dir, os.path.join(os.path.dirname(images_dir), "labels")

    # If no split, attempt to create from flat structure
    flat_images = []
    for dirpath, dirnames, filenames in os.walk(root):
        for fn in filenames:
            if fn.lower().endswith((".jpg",".jpeg",".png",".bmp",".tif",".tiff")):
                flat_images.append(os.path.join(dirpath, fn))
    if len(flat_images) == 0:
        raise RuntimeError("No images found to prepare a split. Please check dataset content.")

    # Create YOLO directories
    img_root = os.path.join(root, "images"); lbl_root = os.path.join(root, "labels")
    for subset in ["train","val","test"]:
        os.makedirs(os.path.join(img_root, subset), exist_ok=True)
        os.makedirs(os.path.join(lbl_root, subset), exist_ok=True)

    # Try to find matching labels (YOLO .txt alongside image names)
    pairs = []
    for img in flat_images:
        base = os.path.splitext(os.path.basename(img))[0]
        label_candidates = glob.glob(os.path.join(os.path.dirname(img), base + ".txt"))
        label = label_candidates[0] if label_candidates else None
        pairs.append((img, label))

    # Split 70/20/10
    img_paths = [p for p,_ in pairs]
    train_imgs, tmp = train_test_split(img_paths, test_size=0.3, random_state=42)
    val_imgs, test_imgs = train_test_split(tmp, test_size=(1/3), random_state=42)  # 0.2/0.1

    def move_pairs(img_list, subset):
        for img in img_list:
            base = os.path.splitext(os.path.basename(img))[0]
            dst_img = os.path.join(img_root, subset, os.path.basename(img))
            shutil.copy2(img, dst_img)
            # Move/copy label if found
            cand = glob.glob(os.path.join(os.path.dirname(img), base + ".txt"))
            if cand:
                shutil.copy2(cand[0], os.path.join(lbl_root, subset, base + ".txt"))

    move_pairs(train_imgs, "train")
    move_pairs(val_imgs,   "val")
    move_pairs(test_imgs,  "test")
    return img_root, lbl_root

# Prepare split (re-entrant safe)
images_dir, labels_dir = ensure_split(work_dir)
print("Images dir:", images_dir)
print("Labels dir:", labels_dir)
print("Subfolders in images:", os.listdir(images_dir))

# Create a minimal data.yaml (1 class: license_plate). Adjust if your dataset has multiple classes.
data_yaml_path = os.path.join(work_dir, "data.yaml")
data_cfg = {
    "train": os.path.join(images_dir, "train"),
    "val":   os.path.join(images_dir, "val"),
    "test":  os.path.join(images_dir, "test"),
    "nc":    1,
    "names": ["license_plate"]
}
with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_cfg, f, sort_keys=False)
print("Wrote data.yaml ->", data_yaml_path)
print(open(data_yaml_path).read())


Images dir: ./license_plate_data/images
Labels dir: ./license_plate_data/labels
Subfolders in images: ['train', 'val', 'test']
Wrote data.yaml -> ./license_plate_data/data.yaml
train: ./license_plate_data/images/train
val: ./license_plate_data/images/val
test: ./license_plate_data/images/test
nc: 1
names:
- license_plate



In [ ]:
# Choose a YOLOv8 model (n: nano, s: small)
model_name = "yolov8n.pt"
epochs = 20
imgsz = 640

model = YOLO(model_name)
results = model.train(data=data_yaml_path, epochs=epochs, imgsz=imgsz, project="runs_lp", name="single_run", exist_ok=True, verbose=True)

print("Training completed.")
print("Run directory:", results.save_dir if hasattr(results, "save_dir") else "See 'runs_lp/detect/single_run'")


Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/license_plate_data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=single_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

In [ ]:

# Validation
val_results = model.val(data=data_yaml_path, imgsz=imgsz, split="val", project="runs_lp", name="single_run_val", exist_ok=True, verbose=True)

# Extract metrics
metrics = val_results.results_dict if hasattr(val_results, "results_dict") else {}
print("Metrics:", metrics)

# Visualize predictions on a few images from the val set
val_images = glob.glob(os.path.join(images_dir, "val", "*.*"))
sample_images = random.sample(val_images, min(6, len(val_images)))

pred_dir = os.path.join("runs_lp", "predictions")
os.makedirs(pred_dir, exist_ok=True)

for img_path in sample_images:
    pred = model.predict(source=img_path, save=True, project=pred_dir, name="vis", exist_ok=True, verbose=False)

print("Saved example predictions to:", os.path.join(pred_dir, "vis"))


Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1261.4±1102.3 MB/s, size: 170.7 KB)
val: Scanning /content/license_plate_data/labels/val.cache... 153 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 153/153 326.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 3.5it/s 2.8s
                   all        153        153      0.993          1      0.994      0.886
Speed: 2.7ms preprocess, 5.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /content/runs_lp/single_run_val
Metrics: {'metrics/precision(B)': 0.9925177318954409, 'metrics/recall(B)': 1.0, 'metrics/mAP50(B)': 0.9944155844155843, 'metrics/mAP50-95(B)': 0.8860900400980102, 'fitness': 0.8860900400980102}
Results saved to /content/runs_lp/predictions/vis
Results saved to /

In [ ]:

# Build a unified list of (image, label) from train+val
trainval_imgs = glob.glob(os.path.join(images_dir, "train", "*.*")) + glob.glob(os.path.join(images_dir, "val", "*.*"))
random.shuffle(trainval_imgs)

def yolo_pair(img_path):
    base = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(labels_dir, "train", base + ".txt")
    if not os.path.exists(label_path):
        label_path = os.path.join(labels_dir, "val", base + ".txt")
    return img_path, (label_path if os.path.exists(label_path) else None)

pairs = [yolo_pair(p) for p in trainval_imgs]
pairs = [p for p in pairs if p[1] is not None]  # keep only those with labels

print(f"Total labeled images for CV: {len(pairs)}")

# Create temp fold directories under work_dir/cv_folds
cv_root = os.path.join(work_dir, "cv_folds")
os.makedirs(cv_root, exist_ok=True)

def write_list_to_file(paths, out_txt):
    with open(out_txt, "w") as f:
        for p in paths:
            f.write(p + "\n")

# Helper to copy images/labels into a YOLO structure
def build_yolo_split(dest_root, train_list, val_list):
    for subset, subset_list in [("train", train_list), ("val", val_list)]:
        img_dest = os.path.join(dest_root, "images", subset)
        lbl_dest = os.path.join(dest_root, "labels", subset)
        os.makedirs(img_dest, exist_ok=True); os.makedirs(lbl_dest, exist_ok=True)
        for img in subset_list:
            base = os.path.splitext(os.path.basename(img))[0]
            # image
            shutil.copy2(img, os.path.join(img_dest, os.path.basename(img)))
            # label (search in train/val labels)
            lbl = os.path.join(labels_dir, "train", base + ".txt")
            if not os.path.exists(lbl):
                lbl = os.path.join(labels_dir, "val", base + ".txt")
            if os.path.exists(lbl):
                shutil.copy2(lbl, os.path.join(lbl_dest, os.path.basename(lbl)))

def run_fold(fold_idx, train_imgs, val_imgs, epochs_per_fold=10):
    fold_root = os.path.join(cv_root, f"fold_{fold_idx}")
    if os.path.exists(fold_root):
        shutil.rmtree(fold_root)
    os.makedirs(fold_root, exist_ok=True)
    build_yolo_split(fold_root, train_imgs, val_imgs)
    # write data.yaml for the fold
    fold_yaml = os.path.join(fold_root, "data.yaml")
    fold_cfg = {
        "train": os.path.join(fold_root, "images", "train"),
        "val":   os.path.join(fold_root, "images", "val"),
        "nc":    1,
        "names": ["license_plate"]
    }
    with open(fold_yaml, "w") as f:
        yaml.safe_dump(fold_cfg, f, sort_keys=False)
    # train
    fold_model = YOLO(model_name)
    fold_results = fold_model.train(data=fold_yaml, epochs=epochs_per_fold, imgsz=imgsz, project="runs_lp", name=f"cv_fold_{fold_idx}", exist_ok=True, verbose=True)
    # validate
    fold_val = fold_model.val(data=fold_yaml, imgsz=imgsz, split="val", project="runs_lp", name=f"cv_fold_{fold_idx}_val", exist_ok=True, verbose=True)
    # collect key metrics
    mdict = fold_val.results_dict if hasattr(fold_val, "results_dict") else {}
    return mdict

# 5-Fold split
kf = KFold(n_splits=5, shuffle=True, random_state=42)
metrics_list = []
imgs_only = [p[0] for p in pairs]

fold_num = 0
for train_idx, val_idx in kf.split(imgs_only):
    train_imgs = [imgs_only[i] for i in train_idx]
    val_imgs   = [imgs_only[i] for i in val_idx]
    mdict = run_fold(fold_num, train_imgs, val_imgs, epochs_per_fold=10)  # adjust epochs for time
    mdict["fold"] = fold_num
    metrics_list.append(mdict)
    fold_num += 1

cv_df = pd.DataFrame(metrics_list)
cv_csv = os.path.join(work_dir, "cv_metrics.csv")
cv_df.to_csv(cv_csv, index=False)
cv_df


Total labeled images for CV: 1679
Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/license_plate_data/cv_folds/fold_0/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=cv_fold_0, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overla

,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),fitness,fold
0,0.979092,0.988095,0.992026,0.835099,0.835099,0
1,0.987505,0.994048,0.994822,0.857772,0.857772,1
2,0.991091,0.993216,0.994669,0.857991,0.857991,2
3,0.984817,0.988095,0.992783,0.849777,0.849777,3
4,1.000000,0.986720,0.994881,0.856350,0.856350,4


In [ ]:

def pretty_summary(df):
    cols = [c for c in df.columns if any(k in c for k in ["precision","recall","map","mAP","mp","mr"])]
    if not cols:
        print("Columns:", df.columns.tolist())
        raise RuntimeError("No metric columns detected; check fold results.")
    summary = df[cols].agg(["mean","std"]).T
    return summary

summary_df = pretty_summary(cv_df)
display(summary_df)

# Save a formatted CSV for your article
summary_path = os.path.join(work_dir, "cv_summary.csv")
summary_df.to_csv(summary_path)
print("Saved CV summary:", summary_path)


,mean,std
metrics/precision(B),0.988501,0.007779
metrics/recall(B),0.990035,0.003344
metrics/mAP50(B),0.993836,0.001336
metrics/mAP50-95(B),0.851398,0.009707


Saved CV summary: /content/license_plate_data/cv_summary.csv


In [ ]:

# Load best weights from single run
best_weights = sorted(glob.glob(os.path.join("runs_lp","detect","single_run","weights","best.pt")), key=lambda p: os.path.getmtime(p))
if len(best_weights)==0:
    # ultralytics uses project/task folder naming; fall back search
    best_weights = glob.glob("runs_lp/**/weights/best.pt", recursive=True)

assert len(best_weights)>0, "Best weights not found. Check training run directories."
best_w = best_weights[-1]

best_model = YOLO(best_w)

test_images = glob.glob(os.path.join(images_dir, "test", "*.*"))
sample_test = random.sample(test_images, min(6, len(test_images))) if len(test_images)>0 else random.sample(glob.glob(os.path.join(images_dir,"val","*.*")), 6)

out_demo = os.path.join("runs_lp","demo_vis")
for img_path in sample_test:
    best_model.predict(source=img_path, save=True, project=out_demo, name="pred", exist_ok=True, verbose=False)

print("Demo predictions saved to:", os.path.join(out_demo, "pred"))


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to /content/runs_lp/demo_vis/pred
Results saved to /content/runs_lp/demo_vis/pred
Results saved to /content/runs_lp/demo_vis/pred
WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Result

*IF THERE ANY ISSUES WITH FILES AFTER DOWNLOADING DATASET*


In [ ]:
# --- Auto-fix data.yaml: find the real images/ and rewrite paths ---
import os, glob, yaml, shutil

# 1) Найдём рабочую директорию (там где распаковали)
candidates = [
    "/kaggle/working/license_plate_data",
    "/content/license_plate_data",
    "./license_plate_data",
    "/content",  # на всякий случай ищем глубже
]
work_dir = next((p for p in candidates if os.path.exists(p)), None)
if work_dir is None:
    raise FileNotFoundError("Не найден work_dir. Укажи вручную путь к рабочей папке с данными.")

print("work_dir:", work_dir)

# 2) Найдём корректную папку images (чтобы внутри были train и val/valid)
def find_images_root(base):
    for root, dirs, files in os.walk(base):
        if os.path.basename(root).lower() == "images":
            subs = set([d.lower() for d in dirs])
            if ("train" in subs) and (("val" in subs) or ("valid" in subs)):
                return root
    return None

images_dir = find_images_root(work_dir)
if images_dir is None:
    # иногда данные попадают на уровень глубже — попробуем внутри возможной вложенной папки с тем же именем
    nested = os.path.join(work_dir, "license_plate_data")
    if os.path.exists(nested):
        images_dir = find_images_root(nested)

if images_dir is None:
    raise RuntimeError("Не удалось найти папку 'images' с подкаталогами train/val. Покажи структуру work_dir.")

print("images_dir:", images_dir)

# 3) Приведём 'valid' -> 'val' и то же для labels
labels_dir = os.path.join(os.path.dirname(images_dir), "labels")
valid_img = os.path.join(images_dir, "valid")
valid_lbl = os.path.join(labels_dir, "valid")
val_img  = os.path.join(images_dir, "val")
val_lbl  = os.path.join(labels_dir, "val")

if os.path.isdir(valid_img) and not os.path.isdir(val_img):
    os.rename(valid_img, val_img)
    print("Renamed images/valid -> images/val")
if os.path.isdir(valid_lbl) and not os.path.isdir(val_lbl):
    os.rename(valid_lbl, val_lbl)
    print("Renamed labels/valid -> labels/val")

# 4) Убедимся, что train/val существуют и непустые
def has_images(p):
    exts = (".jpg",".jpeg",".png",".bmp",".tif",".tiff")
    return any(fn.lower().endswith(exts) for fn in os.listdir(p)) if os.path.isdir(p) else False

assert os.path.isdir(os.path.join(images_dir, "train")), "Нет папки images/train"
assert os.path.isdir(os.path.join(images_dir, "val")),   "Нет папки images/val"

# 5) Если test отсутствует — создадим из части val (необязательно, но полезно)
test_img = os.path.join(images_dir, "test")
test_lbl = os.path.join(labels_dir, "test")
if not os.path.isdir(test_img):
    os.makedirs(test_img, exist_ok=True)
    os.makedirs(test_lbl, exist_ok=True)
    # можно оставить пустым — Ultralytics использует только train/val; test нужен для твоих демо-предсказаний
    print("Created empty images/test and labels/test")

# 6) Перегенерируем data.yaml рядом с work_dir (и обновим переменную data_yaml_path)
data_yaml_path = os.path.join(work_dir, "data.yaml")
cfg = {
    "train": os.path.join(images_dir, "train"),
    "val":   os.path.join(images_dir, "val"),
    "test":  test_img if os.path.isdir(test_img) else os.path.join(images_dir, "val"),
    "nc":    1,
    "names": ["license_plate"],
}
with open(data_yaml_path, "w") as f:
    yaml.safe_dump(cfg, f, sort_keys=False)

print("Rewrote data.yaml:", data_yaml_path)
print(open(data_yaml_path).read())


work_dir: /content/license_plate_data
images_dir: /content/license_plate_data/images
✅ Rewrote data.yaml: /content/license_plate_data/data.yaml
train: /content/license_plate_data/images/train
val: /content/license_plate_data/images/val
test: /content/license_plate_data/images/test
nc: 1
names:
- license_plate

